# Detecting an STR variant in the *APC* gene and predicting its impact
**By Max Verbiest, for the course 'Bioinformatics for Beginners'**  
**contact: max.verbiest@zhaw.ch**

Before working through this notebook, you should have completed the steps outlined in this repo's `README.md` file. I.e., you should have run GangSTR to generate a VCF file of STR genotypes for our sequencing sample.  

In this notebook, we will programatically seach the VCF file to detect an STR variant. Subsequently, we will try to interpret the possible effects of this mutation using the integrative genomics viewer (IGV) web app, Biopython, and Ensembl's variant effect predictor (VEP) tool.

First, let's load some libraries:

In [1]:
import numpy as np
import pandas as pd
from cyvcf2 import VCF
from Bio import SeqIO, Seq

Our first task consists of searching for previously generated VCF file for the STR locus with non-reference alleles. There should only be one such locus. To figure out what information to look for in the VCF file, take some time to browse the [VCF format specification](http://samtools.github.io/hts-specs/VCFv4.4.pdf). Please don't read the whole thing! It should suffice to read the the first few paragraphs and section "1.6.1 Fixed fields". Once you know what the columns `CHROM`, `POS`, `REF`, and `ALT` mean, you are ready to proceed.

For our current project, we can detect the non-reference STR just by checking whether there is an entry in the `ALT` column of the variant: all but one of the variants in our VCF file wille have a `.` in their `ALT` column, except for the non-reference locus. **This is not how you should detect variants in real VCF files.** Typically, variants for multiple samples will be contained in a single VCF file, you would then need to extract the genotype for each sample and compare it to the `REF` and `ALT` columns to determine if a variant is present in a sample or not. For now: we keep it simple by just checking if there is a `.` in the `ALT` column.  There are many ways to approach this task. I will outline two options here, feel free to pick the one that appeals most to you (or try both, or something completely different...).  

One option is a "vanilla" Python file parsing approach: open the VCF file, step through it line by line, and extract the VCF entry of interest. 
Some useful functions for this approach are:  

```python
open()
str.startswith()
str.strip()
str.split()
```

Another approach is to use a VCF file parser that someone else developed. One example of such a library is [cyvcf2](https://brentp.github.io/cyvcf2/) (not availible through the Windows environment). Using the documentation for this library, you should be able to parse the VCF file and extract the entry containing the STR mutation.  

*Note: In general, it is a good idea to use existing libraries for things like file parsing unless you have a very, very good reason not to. Existing libraries that are used by many people can be very robust. Different people will have used them in many different setting, so many bugs will have been noticed and fixed already &mdash; especially in open-source software where everyone can look at the code.*

In [170]:
# Your code goes here




## Inspecting the STR variant in the IGV web app

Open the [IGV web app](https://igv.org/app/) in your browser of choice. It just look like this:

![](images/IGV_empty_example.png)

By default, the human reference genome 'GRCh38' is loaded in the viewer. However, we want to use our custom APC reference sequence. Click on the `Genome` button in the top left, and select `Local File...`. You will then be prompted to select a file to upload. Navigate to the `data/alignments/` folder, and select `APC.fa` **AND** `APC.fa.fai` and the same time, and press `Open`, like so:
![](images/IGV_upload_reference.png)

The IGV web app should now display the reference sequence. You can now inspect the nucleotide sequence if you zoom in a bit, although this is not very interesting by itself:  
![](images/IGV_just_APC.png)

Let's add some of the other information we have available! Press the `Tracks` button at the top, choose `Local File...` again, and &mdash; similarly to how you uploaded the reference genome &mdash; add our sequence alignment (`APC_mut.bam`) + index (`APC_mut.bam.bai`) to the viewer. Then, do the same for the VCF file generated by GangSTR (this file does not need an index). Finally, upload the two files in `data/genome_annotation` (ending in `.gtf.gz` and `.gtf.gz.tbi`). This pair of files is are in [GTF](https://www.ensembl.org/info/website/upload/gff.html) format. This is a file format for annotation genomic sequences. This particular GTF file tells us where the transcript, exons, and coding sequences of the *APC* gene are located. After uploading all information, the final result should look something like what is shown below (once you zoom in a bit). Take a moment to look at the different tracks: do you know what each track represents?
![](images/IGV_complete_example.png)

Now that all our information is loaded into the genome viewer, it's time to take a look at the variant in our sample. Navigate to the location of the STR variant using the coordinates that you got from the GangSTR VCF file in the first part of this notebook (go to `Help`, `Documentation` in the IGV web app and look under `Navigation` if you're having trouble).  

In what type of sequence does the STR variant occur? Do you expect this mutation to have a functional impact on the protein transcribed from this gene? (especially pay attention to the genome annotation track, labelled `APC_canonical_relative_coordinates.gtf.gz`)

## Interpreting the STR variant's impact

So we've identified an STR variant in our sequencing sample, how do we interpret this? Luckily for us, Ensembl has a tool called the [Variant Effect Predictor](https://www.ensembl.org/Tools/VEP). It allows us to upload our identified variant, and the tool will compare it to other resources to predict whether there is a functional impact for our variant. 

You can now paste the first 5 columns of the STR variant from your GangSTR VCF file into the `Input data` field. However, we first need to modify the VCF entry a bit: First, because we used only the *APC* gene sequence in our analysis, the genomic coordinates in our VCF file do not reflect the true coordinates in the reference genome (which is what VEP uses, of course). The *APC* sequence I extracted for this project startrs at chromosome 5, position 112702498. Therefore, to convert from our coordinate system to GRCh38 coordinates, we just need to add 112702498. Additionally, VEP expects chromosome identifiers just as a number, not prefixed with `chr`, as we currently have them. 

So, for example, if this is the VCF entry for the STR variant you identified:
```
chr5	89325	.	ctgctgctgctgctg	ctgctgctg
```

You should modify it to look like this:
```
5	112791823	.	ctgctgctgctgctg	ctgctgctg
```

Once your variant is in the correct format, paste it into the `Input data` field, and press `Run >` at the bottom of the page. After a moment, you will be taken to this page:  

![](images/VEP_queued_job.png)

Once your job status changes from `Queued` to `Done`, click `[View results]` to explore what information VEP found for our STR variant. Take some time to explore the output, and follow the links VEP provides to get additional information on the STR variant. Do you expect the variant we identified to be impactful? What sort of phenotype do you expect the variant to be involved with?

## Optional: manual prediction of STR variant impact

The VEP tool is a great way to get an idea of the possible effects of a detected variant. However, based on the fact that the STR variant occurs in the coding sequence of exon 16 of the *APC* gene, and that the variant consists of a deletion of a number of nucleotides that is not a multiple of three, you could have already suspected that the variant results in a frameshift event.

Frameshifts usually lead to shorter transcripts because of the high probability of introducing a premature stop codon.  As an optional challenge, you can try to write some Python code to compare the longest possible open reading frame (ORF) between the wild-type *APC* exon 16, and the *APC* exon 16 with our STR variant. The wild-type *APC* sequence is the reference sequence we have been using for this project: `data/reference/APC.fa`, the variant sequence was generated from the alignment: `results/APC_mut_consensus.fa`.

I found the [Biopython](https://biopython.org/) package to be very useful for this task, especially the [SeqIO](https://biopython.org/wiki/SeqIO) interface and the [Seq](https://biopython.org/wiki/Seq) class. But, as always, feel free to use whatever approach you prefer. According to my analysis, the wild-type exon 16 contains an ORF of 6570 nucleotides (2190 amino acids), whereas the longest ORF in *APC* exon 16 with our STR variant is 2451 nucleotides long (817 amino acids).

In [180]:
# Your code goes here


